In [2]:
import pandas as pd 
import numpy as numpy
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [3]:
pd.set_option('display.max_columns' , None)
df = pd.read_csv("IMDB_Top250movies2_OMDB_Detailed.csv")
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [4]:
#data preprocessing 

In [5]:
df['plot'] = df['Plot'].str.lower()
#convert lower case and remove extra space and punc.
df['plot'] = df['plot'].apply(lambda x: re.sub('[^a-z A-Z]',' ',x))
df['plot'] = df['plot'].apply(lambda x: re.sub('\s+',' ',x))
df['plot']

0      two imprisoned men bond over a number of years...
1      the aging patriarch of an organized crime dyna...
2      the early life and career of vito corleone in ...
3      when the menace known as the joker emerges fro...
4      a jury holdout attempts to prevent a miscarria...
                             ...                        
245    blacksmith will turner teams up with eccentric...
246    a former child star torments her paraplegic si...
247    travis henderson an aimless drifter who has be...
248    desperate measures are taken by a man who trie...
249    a stranger in the city asks questions no one h...
Name: plot, Length: 250, dtype: object

In [6]:
#convert sentence to words 
df['plot'] = df['plot'].apply(lambda x: nltk.word_tokenize(x))
df['plot']

0      [two, imprisoned, men, bond, over, a, number, ...
1      [the, aging, patriarch, of, an, organized, cri...
2      [the, early, life, and, career, of, vito, corl...
3      [when, the, menace, known, as, the, joker, eme...
4      [a, jury, holdout, attempts, to, prevent, a, m...
                             ...                        
245    [blacksmith, will, turner, teams, up, with, ec...
246    [a, former, child, star, torments, her, parapl...
247    [travis, henderson, an, aimless, drifter, who,...
248    [desperate, measures, are, taken, by, a, man, ...
249    [a, stranger, in, the, city, asks, questions, ...
Name: plot, Length: 250, dtype: object

In [7]:
#remove the stopping words such as to, of, a,etc..
cw = nltk.corpus.stopwords.words('english')
temp_cw = []
for sen in df['plot']:
    tmp = []
    for word in sen:
        if word not in cw and len(word) >= 3:
            tmp.append(word)
    temp_cw.append(tmp)
temp_cw

',
  'still',
  'haunt',
  'decades',
  'later'],
 ['young', 'boy', 'raised', 'within', 'confines', 'small', 'shed'],
 ['settling',
  'differences',
  'japanese',
  'pow',
  'camp',
  'commander',
  'british',
  'colonel',
  'operates',
  'oversee',
  'men',
  'construction',
  'railway',
  'bridge',
  'captors',
  'oblivious',
  'plan',
  'allies',
  'destroy'],
 ['blade',
  'runner',
  'must',
  'pursue',
  'try',
  'terminate',
  'four',
  'replicants',
  'stole',
  'ship',
  'space',
  'returned',
  'earth',
  'find',
  'creator'],
 ['unconfident',
  'young',
  'woman',
  'cursed',
  'old',
  'body',
  'spiteful',
  'witch',
  'chance',
  'breaking',
  'spell',
  'lies',
  'self',
  'indulgent',
  'yet',
  'insecure',
  'young',
  'wizard',
  'companions',
  'legged',
  'walking',
  'castle'],
 ['jazz', 'pianist', 'falls', 'aspiring', 'actress', 'los', 'angeles'],
 ['twins',
  'journey',
  'middle',
  'east',
  'discover',
  'family',
  'history',
  'fulfill',
  'mother',
  'last',

In [8]:
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response,plot
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True,"[two, imprisoned, men, bond, over, a, number, ..."
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True,"[the, aging, patriarch, of, an, organized, cri..."
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True,"[the, early, life, and, career, of, vito, corl..."
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True,"[when, the, menace, known, as, the, joker, eme..."
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True,"[a, jury, holdout, attempts, to, prevent, a, m..."


In [9]:
df['Genre'] = df['Genre'].apply(lambda x: x.split(','))
df['Actors'] = df['Actors'].apply(lambda x: x.split(',')[:4])
df['Director'] = df['Director'].apply(lambda x: x.split(','))

In [10]:
def clean(sen):
    #temprory list to output 
    tmp = []
    for x in sen:
        tmp.append(x.lower().replace(' ',''))
    return tmp

df['Genre'] = [clean(x) for x in df['Genre']]
df['Actors'] = [clean(x) for x in df['Actors']]
df['Director'] = [clean(x) for x in df['Director']]


In [11]:
df['Actors'][0]

['timrobbins', 'morganfreeman', 'bobgunton', 'williamsadler']

In [12]:
#combine all the colomun data as one for each moive 

coloumns = ['plot','Genre','Actors','Director']
list_data =[]
for i in range(len(df)):
    word = ''
    for c in coloumns:
        word += ' '.join(df[c][i]) + ' '
    list_data.append(word)
list_data

uddies to help get it comedy crime jeffbridges johngoodman juliannemoore stevebuscemi joelcoen ethancoen ',
 'a boy who communicates with spirits that don t know they re dead seeks the help of a disheartened child psychologist drama mystery thriller brucewillis haleyjoelosment tonicollette oliviawilliams m.nightshyamalan ',
 'a research facility in antarctica comes across an alien force that can become anything it touches with accuracy the members must now find out who s human and who s not before it s too late horror mystery sci-fi kurtrussell wilfordbrimley t.k.carter davidclennon johncarpenter ',
 'after his son is captured in the great barrier reef and taken to sydney a timid clownfish sets out on a journey to bring him home animation adventure comedy albertbrooks ellendegeneres alexandergould willemdafoe andrewstanton leeunkrich ',
 'an old couple visit their children and grandchildren in the city but the children have little time for them drama chishûryû chiekohigashiyama setsuko

In [13]:
df['clean_input'] = list_data
df = df[['Title','clean_input']]
df.head()


,Title,clean_input
0,The Shawshank Redemption,two imprisoned men bond over a number of years...
1,The Godfather,the aging patriarch of an organized crime dyna...
2,The Godfather: Part II,the early life and career of vito corleone in ...
3,The Dark Knight,when the menace known as the joker emerges fro...
4,12 Angry Men,a jury holdout attempts to prevent a miscarria...


In [14]:
# feature Extraction


In [28]:
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(df['clean_input'])
pickle.dump(features,open('features.pkl','wb'))

In [27]:
# create cosince similarity matrix 
cos_sim = cosine_similarity(features,features)
print(cos_sim)


[[1.         0.04407871 0.04092886 ... 0.05127254 0.03390218 0.01569978]
 [0.04407871 1.         0.20073959 ... 0.04863452 0.09035088 0.02674638]
 [0.04092886 0.20073959 1.         ... 0.08559317 0.09133632 0.04873865]
 ...
 [0.05127254 0.04863452 0.08559317 ... 1.         0.06693595 0.04809861]
 [0.03390218 0.09035088 0.09133632 ... 0.06693595 1.         0.01895116]
 [0.01569978 0.02674638 0.04873865 ... 0.04809861 0.01895116 1.        ]]


In [17]:
# moive recommendation
index = pd.Series(df['Title'])
index.head()

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [29]:
def recommend_moive(title):
    movies = []
    #map index with title
    idx = index [index == title].index[0]
    print(idx)
    #calculate similarity cost 
    score = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    #top 10 in score
    top10 = list(score.iloc[1:11].index)
    #print(top10)

    #add moive to the list 
    for i in top10:
        movies.append(df['Title'][i])
    return movies


In [30]:
recommend_moive('The Wolf of Wall Street')


153


['Goodfellas',
 'Raging Bull',
 'The Godfather: Part II',
 'Shutter Island',
 'Spotlight',
 'Batman Begins',
 'The Departed',
 'The Dark Knight',
 'The Apartment',
 'Catch Me If You Can']

In [33]:
pickle.dump(df,open('moives.pkl','wb'))


In [25]:
df['Title']

0                               The Shawshank Redemption
1                                          The Godfather
2                                 The Godfather: Part II
3                                        The Dark Knight
4                                           12 Angry Men
                             ...                        
245    Pirates of the Caribbean: The Curse of the Bla...
246                     What Ever Happened to Baby Jane?
247                                         Paris, Texas
248                                             Drishyam
249                                                   PK
Name: Title, Length: 250, dtype: object